In [ ]:
## Used for feature extraction

In [ ]:
import os
import pandas as pd
from pyAudioAnalysis import MidTermFeatures as mtf
import numpy as np

# Input is the file where the audios are located, used to train different composers and categories(Mozart-Piano,Bethoveen-Symphonies etc)
mid_term_features, wav_file_list, mid_feature_names = mtf.directory_feature_extraction('C:/Users/user/OneDrive/Υπολογιστής/Εργασία machine learning dataset/Mozart/Piano Pieces/Testing', 1, 1, 0.1, 0.1)
m = mid_term_features.mean(axis=0)
s = np.std(mid_term_features, axis = 0)
mid_term_features_2 = (mid_term_features - m) / s

# dataFrame from the features and file names
df = pd.DataFrame(mid_term_features_2, columns=mid_feature_names)


# dataFrame to CSV 
df.to_csv('Mozart_piano_testing', index=False)



In [ ]:
## Used for merging the produced dataframe and adding labels to them

In [ ]:
import pandas as pd

# Read CSV
df1 = pd.read_csv("Schubert_piano_training")
df2 = pd.read_csv("Schumann_piano_training")
df3 = pd.read_csv("Mozart_piano_training")
df4 = pd.read_csv("Bethoveen_piano_training")

# New column with the labels
df1["label"] = 1
df2["label"] = 2
df3["label"] = 3
df4["label"] = 4

# Merge datafram
merged_df = pd.concat([df1, df2, df3, df4])

# Save merged to CSV 
merged_df.to_csv("Piano", index=False)

# Same for "testing" 
df5 = pd.read_csv('Schubert_piano_testing')
df6 = pd.read_csv('Schumann_piano_testing')
df7 = pd.read_csv('Mozart_piano_testing')
df8 = pd.read_csv('Bethoveen_piano_testing')

df5["label"] = 1
df6["label"] = 2
df7["label"] = 3
df8["label"] = 4

merged_df_2 = pd.concat([df5, df6, df7, df8])
merged_df_2.to_csv("Piano_testing", index=False)



In [ ]:
## Used for training and input of testing

In [15]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pandas as pd

# Read training data
train_data = pd.read_csv("Piano")

# Define the features and target
X = train_data.drop("label", axis=1)
y = train_data["label"]

# SelectKBest to select the top N features
selector = SelectKBest(mutual_info_classif, k=138)
selector.fit(X, y)

# Get the selected features
selected_features = selector.get_support(indices=True)

# Print names of features
print(X.columns[selected_features])

# test data
test_data = pd.read_csv("Piano_testing")

# Drop labels in x, y are test labels
X_test = test_data.drop("label", axis=1)
Y_test = test_data["label"]

# Select only the features that were selected in the previous step for test and trainign
selected_features_name = X.columns[selected_features]
X = X[selected_features_name]
X_test = X_test[selected_features_name]

# Train a classifier on the training data
#clf = RandomForestClassifier()
#clf = LogisticRegression()
clf = SVC()
#clf = KNeighborsClassifier(n_neighbors=4)
#clf = MLPClassifier(max_iter=5000)
#clf = GaussianNB()

clf.fit(X, y)

#K-fold validation
cv = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)
p1 = cross_val_score(clf, X, y, cv=cv, scoring='f1_macro')


print(f"mean f1: {p1.mean():.3f}, sigma f1: {p1.std():.3f}, 95% conf: {p1.mean()-2*p1.std():.3f} - {p1.mean()+2*p1.std():.3f}")











Index(['zcr_mean', 'energy_mean', 'energy_entropy_mean',
       'spectral_centroid_mean', 'spectral_spread_mean',
       'spectral_entropy_mean', 'spectral_flux_mean', 'spectral_rolloff_mean',
       'mfcc_1_mean', 'mfcc_2_mean',
       ...
       'delta chroma_6_std', 'delta chroma_7_std', 'delta chroma_8_std',
       'delta chroma_9_std', 'delta chroma_10_std', 'delta chroma_11_std',
       'delta chroma_12_std', 'delta chroma_std_std', 'bpm', 'ratio'],
      dtype='object', length=138)
mean f1: 0.774, sigma f1: 0.013, 95% conf: 0.748 - 0.801


In [11]:
## Used for testing

In [16]:
import numpy as np
Y_pred= clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ", accuracy)



Accuracy of the model:  0.387
